In [134]:
import os
#os.listdir('/home/em-student/experiment_boom/arch_cov_1/leaks/sim_input/')
#os.listdir('/home/em-student/experiment_1/ct_cov_1/leaks/sim_input/')
os.listdir('/home/em-student/experiment_boom/ctb_cov_eq_50_1/leaks/sim_input/')


['id_574.si',
 'id_901.si',
 'id_1795.si',
 'id_574.cov',
 'id_1795.cov',
 'id_901.cov']

In [135]:
# input files
#in_file = '/home/em-student/experiment_1/ct_cov_1/leaks/sim_input/id_606718.si'
#in_file = '/home/em-student/experiment_boom/arch_cov_1/leaks/sim_input/id_8536.si'
in_file = '/home/em-student/experiment_boom/ctb_cov_eq_50_1/leaks/sim_input/id_574.si'


In [136]:
# config
import tempfile
from src.utils import *
from Config import ROCKET_CONF, BOOM_CONF
outdir = tempfile.mkdtemp()


(mutator, preprocessor, hscHost) = setupHSC('Template', outdir, proc_num=0, debug=True, contract='ct', isa='RV64IM')

(sim_input, (data_a, data_b), assert_intr) = mutator.read_siminput(in_file)

(hsc_input, rtl_input, symbols) = preprocessor.process(sim_input, data_a, data_b, False)
stop = [ proc_state.NORMAL ]

In [137]:
# process and run
args = [ hscHost.sail ] + hscHost.sail_args + ['-vplatform', '-vinstr']
args_a = args + [ hsc_input.binary_a, '-o', hscHost.out_a]
result = subprocess.run(args_a, stdout=subprocess.PIPE)

# Get the stdout and stderr from the result
stdout = result.stdout.decode()

[Sail] Allocating new block 0x80000000
[Sail] Allocating new block 0x0

















































































































































































































































































































































































In [138]:
print(stdout)

using ct as hardware-software contract.
Running file /tmp/tmp8mv_k7qy/.input_0_a.elf.
SEQ-CT
[0] [M]: 0x0000000000001000 (0x00000297) auipc t0, 0

[1] [M]: 0x0000000000001004 (0x02028593) addi a1, t0, 32

[2] [M]: 0x0000000000001008 (0xF1402573) csrrs a0, mhartid, zero

[3] [M]: 0x000000000000100C (0x0182B283) ld t0, 24(t0)

[4] [M]: 0x0000000000001010 (0x00028067) jalr zero, t0, 0

[5] [M]: 0x0000000080000000 (0x00010F97) auipc t6, 16

[6] [M]: 0x0000000080000004 (0x000F8F93) addi t6, t6, 0

[7] [M]: 0x0000000080000008 (0x000FB083) ld ra, 0(t6)

[8] [M]: 0x000000008000000C (0x008FB103) ld sp, 8(t6)

[9] [M]: 0x0000000080000010 (0x010FB183) ld gp, 16(t6)

[10] [M]: 0x0000000080000014 (0x018FB203) ld tp, 24(t6)

[11] [M]: 0x0000000080000018 (0x020FB283) ld t0, 32(t6)

[12] [M]: 0x000000008000001C (0x028FB303) ld t1, 40(t6)

[13] [M]: 0x0000000080000020 (0x030FB383) ld t2, 48(t6)

[14] [M]: 0x0000000080000024 (0x038FB403) ld fp, 56(t6)

[15] [M]: 0x0000000080000028 (0x040FB483) ld s1, 64

In [139]:
fb = open(os.path.join(outdir, '.input_0_a.S'))
fb_lines = fb.readlines()
for l in fb_lines:
    print(l)

# Assembly file template for fuzzing



#*********************************************************************

# test_template.S

#---------------------------------------------------------------------



#include "riscv_test.h"

#include "test_macros.h"

#include "fuzz_test.h"



#define TESTEND gp



RVTEST_RV64UF



// RVTEST_CODE_BEGIN

        .section .text.init;

        .align  6;

        .weak stvec_handler;

        .weak mtvec_handler;

        .global _start;

_start:

        /* reset vector */

INIT_XREGS

        j reset_vector;

        .align;

init_freg:

INIT_FREGS

        ret

        .align;

trap_stvec:

        clear_sie;

_fuzz_suffix:

_s0:    sraw x21, x10, x26                        ;

_s1:    la x8, d_3_19                             ;

        sb x23, -24(x8)                           ;

_s2:    bgeu x21, x12, _s3                        ;

_s3:    srai x9, x13, 29                          ;

_s4:    fence                                     ;

_s5:;

    

In [140]:
hscHost.sail_args

['-V', '-L', 'ct']

In [141]:
os.listdir(outdir)

['.input_0_a.elf',
 '.input_0.hex',
 '.input_0.symbols',
 '.input_0_a.S',
 '.input_0_b.elf',
 '.hsc_out_0_a.txt',
 '.input_0_b.S',
 '.input_0.si']

In [142]:
import re

def extract_X(string):
    pattern = r'^handling exc#0x0((?!8|9|b|B)\w)'
    matches = re.findall(pattern, string, re.MULTILINE)
    return matches


In [143]:
extract_X(stdout)

[]

In [144]:
list(map(lambda x: ord(x), ['7', '8', '9', 'A', 'b']))

[55, 56, 57, 65, 98]